In [17]:
with open ("test2.csv", "r") as t1:
    dict_SNPs={} ###Calculate major, minor, and wildtypes by going through each line
    for lines in t1: ##Add number of major, minor, 
        #print(lines)
        currentVAF=0
        if "Sequence Name" not in lines:
            #print(lines)
            if "->" in lines.split(",")[7]:
                currentVAF=float(lines.split(",")[7][0:4].strip("%"))
            else:
                currentVAF=float(lines.split(",")[7].strip("%"))
            #if "S108N" in lines:
             #   print(currentVAF)
             #   print(lines)
            if currentVAF >=50:
                #print(lines.split(",")[9])
                #if "S108N" in lines:
                #    print(currentVAF)
                #    print(lines)
                if (lines.split(",")[1],"Major",lines.split(",")[9]) not in dict_SNPs:
                    dict_SNPs[lines.split(",")[1],"Major",lines.split(",")[9]]=1
                else:
                    dict_SNPs[lines.split(",")[1],"Major",lines.split(",")[9]]=dict_SNPs[lines.split(",")[1],"Major",lines.split(",")[9]]+1
            if 0<currentVAF<50:
                if (lines.split(",")[1],"Minor",lines.split(",")[9]) not in dict_SNPs:
                    dict_SNPs[lines.split(",")[1],"Minor",lines.split(",")[9]]=1
                else:
                    dict_SNPs[lines.split(",")[1],"Minor",lines.split(",")[9]]=dict_SNPs[lines.split(",")[1],"Minor",lines.split(",")[9]]+1
            if currentVAF ==0:
                if (lines.split(",")[1],"Wildtype",lines.split(",")[9]) not in dict_SNPs:
                    dict_SNPs[lines.split(",")[1],"Wildtype",lines.split(",")[9]]=1
                else:
                    dict_SNPs[lines.split(",")[1],"Wildtype",lines.split(",")[9]]=dict_SNPs[lines.split(",")[1],"Wildtype",lines.split(",")[9]]+1

with open ("test2.csv", "r") as t1:
    dict_SNPs_minus={} ###Calculate major, minor, and wildtypes by going through each line. Subtract the last letter from the SNP to cover several mutations.
    for lines in t1:
        #print(lines)
        currentVAF=0
        if "Sequence Name" not in lines:
            if "->" in lines.split(",")[7]:
                currentVAF=float(lines.split(",")[7][0:4].strip("%"))
            else:
                currentVAF=float(lines.split(",")[7].strip("%"))
            if currentVAF >=50:
                #print(lines.split(",")[9])
                if (lines.split(",")[1],"Major",lines.split(",")[9][0:-1]) not in dict_SNPs_minus:
                    dict_SNPs_minus[lines.split(",")[1],"Major",lines.split(",")[9][0:-1]]=1
                else:
                    #if "S108N" in lines:
                    #    print(lines)
                    #    print(dict_SNPs_minus)
                    dict_SNPs_minus[lines.split(",")[1],"Major",lines.split(",")[9][0:-1]]=dict_SNPs_minus[lines.split(",")[1],"Major",lines.split(",")[9][0:-1]]+1
            if 0<currentVAF<50:
                if (lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]) not in dict_SNPs_minus:
                    dict_SNPs_minus[lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]]=1
                else:
                    dict_SNPs_minus[lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]]=dict_SNPs_minus[lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]]+1
            if currentVAF ==0:
                if (lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]) not in dict_SNPs_minus:
                    dict_SNPs_minus[lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]]=1
                else:
                    dict_SNPs_minus[lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]]=dict_SNPs_minus[lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]]+1

dict_SNPs_SUM={} ###Find sum of major, minor, wildtypes, total  samples
dict_minus_test={}

#for items in dict_SNPs:
#    if "C469" in items[2]:
#        print(dict_SNPs[items])

dict_SNPs_2={} ##The purpose of adding wildtype to second mutations
for items in dict_SNPs:
    if (items[0], "Wildtype", items[2]) not in dict_SNPs:
        #print(items)
        dict_SNPs_2[items[0], "Wildtype", items[2]]=dict_SNPs_minus[items[0],"Wildtype",items[2][0:-1]]

#print(dict_SNPs)
#print(dict_SNPs['PfCRT', 'Wildtype', 'N75E'])
for items in dict_SNPs_2: ##Add back to the original dictionary
    #print(items)
    #print(dict_SNPs_2[items])
    dict_SNPs[items]=dict_SNPs_2[items]
    

for items in dict_SNPs:
    dict_SNPs_SUM[items[0],items[2]]=0
    

dict_count={} ## dictionary for couting how many times dict_SNP_minus which is without last letter of G_Annotation is added.
for items in dict_SNPs:
    #print(items)
    #if "N75" in items[2]:
    #    print(items)
    if (items[0],items[2]) in dict_count and items[1]=="Wildtype":
        dict_count[items[0],items[2]]=1
    if (items[0],items[2]) not in dict_count and items[1]=="Wildtype":
        dict_count[items[0],items[2]]=0
    if items[1]=="Wildtype":
        if dict_count[items[0],items[2]]<1:
            dict_SNPs_SUM[items[0],items[2]]=dict_SNPs_SUM[items[0],items[2]]+dict_SNPs[items]
        #if dict_count[items[0],items[2][0:-1]]==1:
        #    print(items)
    if items[1]!="Wildtype":
        dict_SNPs_SUM[items[0],items[2]]=dict_SNPs_SUM[items[0],items[2]]+dict_SNPs[items]

#print(dict_SNPs_SUM)        
#print(dict_SNPs_SUM_up)
        
dict_SNPS_ratio = {} ##Find the ratio of major, minor, wildtypes in total samples
for items in dict_SNPs:
    #if items[1]=="Wildtype":
    #    dict_SNPS_ratio[items]=round(dict_SNPs_minus[items[0],items[1],items[2][0:-1]]/dict_SNPs_SUM_up[items[0],items[2]],5)
    #else:
    #print(items)
    #print(dict_SNPs_SUM[items[0],items[2]])
    dict_SNPS_ratio[items]=round(dict_SNPs[items]/dict_SNPs_SUM[items[0],items[2]],5)

#print(dict_SNPS_ratio)

dict_SNPs_final = {} ##Create final form for the SNPs
for items in dict_SNPs:
    #print(items)
    #print(items[2])
    #print(items[0])
    #if items[0] == "DHPS_437Corrected " and items[2]== "A581V":
    #    print(items)
    #    print(dict_SNPS_ratio[items])
    if (items[0],items[2]) not in dict_SNPs_final:
        dict_SNPs_final[items[0],items[2]]=[0,0,0,0]
    if items[1]=="Wildtype":
        dict_SNPs_final[items[0],items[2]][0]=dict_SNPS_ratio[items]
    if items[1]=="Major":
        dict_SNPs_final[items[0],items[2]][1]=dict_SNPS_ratio[items]
    if items[1]=="Minor":
        dict_SNPs_final[items[0],items[2]][2]=dict_SNPS_ratio[items]
    dict_SNPs_final[items[0],items[2]][3]=dict_SNPs_SUM[items[0],items[2]]

#print(dict_SNPs_final)
with open ("Tab_Table_SNP.csv", "w") as t2:
    t2.write("Gene+SNP,Wildtype: AF=0%,Minor: AF < 50%,Major: AF >= 50%,Samplesize\n")
    for items in dict_SNPs_final:
        t2.write(items[0]+":"+items[1]+","+str(dict_SNPs_final[items][0])+","+str(dict_SNPs_final[items][1])+","+str(dict_SNPs_final[items][2])+","+"N="+str(dict_SNPs_final[items][3])+"\n")
    

